In [1]:
%cd ..

/home/jovyan/working


In [2]:
import math
import os

import pandas as pd
import torch

In [3]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [4]:
from omegaconf import OmegaConf

omega_conf_dir = "./config"

c_main = OmegaConf.load(os.path.join(omega_conf_dir, "main.yaml"))
c_data = OmegaConf.load(os.path.join(omega_conf_dir, "data.yaml"))
c = OmegaConf.merge(c_main, c_data)

c.wandb.enabled = False

In [5]:
from src.ice_cube.data_loader import (
    collate_fn_minus_minus,
    collate_fn_minus_plus,
    collate_fn_plus_minus,
    make_test_dataloader,
)
from src.ice_cube.model import load_pretrained_model
from src.ice_cube.scoring import angular_dist_score
from src.ice_cube.submission import to_submission_df

graphnet: INFO     2023-03-19 13:57:34 - get_logger - Writing log to logs/graphnet_20230319-135734.log


2023-03-19 13:57:34,916 [INFO] [logging] Writing log to logs/graphnet_20230319-135734.log


graphnet: WARNING  2023-03-19 13:57:34 - warn_once - `icecube` not available. Some functionality may be missing.


2023-03-19 13:57:34,924 [WARNING] [logging] `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-03-19 13:57:36.230482: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 13:57:36.370397: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set th

## SQLiteDataset

In [6]:
database_path = os.path.join(c.data.dir.dataset, "test_db.db")
dataloader = make_test_dataloader(c, database_path)

graphnet: WARNING  2023-03-19 13:57:39 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth


2023-03-19 13:57:39,738 [WARNING] [logging] Removing the following (missing) truth variables: zenith, azimuth


## Test Data

In [7]:
model = load_pretrained_model(c, dataloader, state_dict_path=c.inference_params.model_path)

2023-03-19 13:57:39,749 [INFO] [instantiator] Created a temporary directory at /tmp/tmp712b9u0u
2023-03-19 13:57:39,750 [INFO] [instantiator] Writing /tmp/tmp712b9u0u/_remote_module_non_sriptable.py


In [8]:
results = model.predict(
    gpus=[0],
    dataloader=dataloader,
    #prediction_columns=model.prediction_columns,
    #additional_attributes=[c.settings.index_name],
)

2023-03-19 13:57:39,929 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-03-19 13:57:39,931 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-03-19 13:57:39,932 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-03-19 13:57:39,932 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-03-19 13:57:39,936 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-03-19 13:57:39,937 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-03-19 13:57:39,938 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-03-19 13:57:39,938 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-03-19 13:57:44,114 [INFO] [cuda] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting: 0it [00:00, ?it/s]

In [9]:
predictions = (
    torch.cat(results, dim=1).detach().cpu().numpy()
)

In [10]:
predictions

array([[ 9.9333233e-01,  4.9131688e-02, -1.0429241e-01,  2.8534021e+00],
       [-5.6608284e-01, -2.3402302e-01, -7.9043239e-01,  7.8869848e+00],
       [-1.7325106e-01, -9.8453736e-01,  2.5886854e-02,  7.9658882e+01]],
      dtype=float32)

In [12]:
for i in dataloader:
    log.info(i.get_example(1).features)
    log.info(i.get_example(1).x.shape)
    log.info(i.get_example(1).x[0])
    log.info(i.get_example(1).y[0])
    log.info(i.get_example(1).z[0])
    log.info(i.get_example(1).time[0])
    log.info(i.get_example(1).charge[0])
    # log.info(i.get_example(1).to_namedtuple())
    break

2023-03-19 13:58:11,732 [INFO] [3411257887] ['x', 'y', 'z', 'time', 'charge', 'auxiliary']
2023-03-19 13:58:11,735 [INFO] [3411257887] torch.Size([35, 6])
2023-03-19 13:58:11,737 [INFO] [3411257887] tensor([-1.0970e+01,  6.7200e+00, -4.2333e+02,  5.9290e+03,  1.0250e+00,
         1.0000e+00])
2023-03-19 13:58:11,740 [INFO] [3411257887] tensor(6.7200)
2023-03-19 13:58:11,743 [INFO] [3411257887] tensor(-423.3300)
2023-03-19 13:58:11,746 [INFO] [3411257887] tensor(5929.)
2023-03-19 13:58:11,748 [INFO] [3411257887] tensor(1.0250)
